<a href="https://colab.research.google.com/github/MartinKugler97/PLN/blob/main/1_clasificacion_grupo_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# APARTADO 1 - CLASIFICACIÓN DE TEXTOS
### Autores: Diego Recover y Martín Kugler (Grupo 09)

In [ ]:
# !pip install ydata_profiling
# !pip install contractions
# !pip install emoji

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re
import contractions
import emoji
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

ModuleNotFoundError: No module named 'contractions'

In [ ]:
# nltk.download('punkt')
# nltk.download('punkt_tab')
# nltk.download('stopwords')
# nltk.download('wordnet')

## TAREA 1: Preprocesamiento del dataset.

In [ ]:
# El fichero csv proporcionado se encuentra localizado en un repositorio de GitHub público al que accederemos:
path = 'https://raw.githubusercontent.com/MartinKugler97/PLN/main/dataset_reddit.csv'

# Leemos el fichero csv mediante el método read_csv de pandas:
df = pd.read_csv(path)

# Inspeccionamos la estructura del dataset:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8989 entries, 0 to 8988
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      8989 non-null   int64 
 1   label           8989 non-null   int64 
 2   comment         8975 non-null   object
 3   author          8989 non-null   object
 4   subreddit       8989 non-null   object
 5   score           8989 non-null   int64 
 6   ups             8989 non-null   int64 
 7   downs           8989 non-null   int64 
 8   date            8989 non-null   object
 9   created_utc     8989 non-null   object
 10  parent_comment  8989 non-null   object
dtypes: int64(5), object(6)
memory usage: 772.6+ KB


In [ ]:
# A partir de esto, debemos decidir qué columnas y qué valores nos interesan:

# Para una primera aproximación únicamente tendremos en cuenta
# las columnas comment y subreddit. Más adelante, en caso de querer profundizar para
# conseguir un mejor resultado.
df = df[['comment', 'subreddit']]
df

,comment,subreddit
0,best wall-music experience ever going see roge...,Christianity
1,"tend agree you, puppeteers 'persuasive'.",conspiracy
2,zionist overlords say is.,uncensorednews
3,"seriously, like wwe...",worldpolitics
4,nobody cares.... trump won.. go spam sub bs.,The_Donald
...,...,...
8984,"pressure sarah palin, also let rahm emanuel go.",politics
8985,would thought volcanoes might erupt serious di...,politics
8986,thank god geneva convention 16 acts torture th...,politics
8987,video happen.,politics


In [ ]:
# Analizamos también si hay valores NaN:
df[df['subreddit'].isna()] # En el caso de 'subreddit' no hay.
df[df['comment'].isna()] # Pero sí los hay en 'comment'

# Eliminaremos estas filas, pues no nos proporcionan información alguna:
df.dropna(subset=['comment'], inplace=True)

### Normalización del dataset:

#### 1) Case normalization:

In [ ]:
df['comment'] = df['comment'].str.lower()

#### 2) Eliminación de emoticonos, espacios extra y URLs:

In [ ]:
# Eliminación de URLs:

url_pattern = re.compile(r'http[s]?://\S+')
df['comment'] = df['comment'].apply(lambda x: re.sub(r'http[s]?://\S+', '', str(x), flags=re.MULTILINE))

# Eliminación de espacios extra y contracciones:

df['comment'] = df['comment'].apply(lambda x: re.sub(r'\s+', ' ', str(x)).strip())
df['comment'] = df['comment'].apply(lambda x: contractions.fix(str(x)))

# Eliminación de emoticonos:
df['comment'] = df['comment'].apply(lambda x: emoji.demojize(str(x)))

#### 3) Eliminación de puntuación:

In [ ]:
df['comment'] = df['comment'].str.translate(str.maketrans('', '', string.punctuation))

#### 4) Tokenización:

In [ ]:
df['comment'] = df['comment'].apply(nltk.word_tokenize)

#### 5) Eliminación de stopwords:

In [ ]:
stop_words = set(stopwords.words('english'))
df['comment'] = df['comment'].apply(lambda tokens: [t for t in tokens if t not in stop_words])

#### 6) Lematización:

In [ ]:
# Elegimos lematización sobre stemming al ser más preciso:
lemmatizer = WordNetLemmatizer()
df['comment'] = df['comment'].apply(lambda tokens: [lemmatizer.lemmatize(t) for t in tokens])

In [ ]:
# Finalmente, volvemos a juntar las palabras en un único string:
df['comment'] = df['comment'].apply(lambda x: ' '.join(x))

In [ ]:
# Así quedaría finalmente el DataFrame después del preprocesado:
df

,comment,subreddit
0,best wallmusic experience ever going see roger...,Christianity
1,tend agree puppeteer persuasive,conspiracy
2,zionist overlord say,uncensorednews
3,seriously like wwe,worldpolitics
4,nobody care trump go spam sub b,The_Donald
...,...,...
8984,pressure sarah palin also let rahm emanuel go,politics
8985,would thought volcano might erupt serious dise...,politics
8986,thank god geneva convention 16 act torture thr...,politics
8987,video happen,politics


## TAREA 2: Aplicación de modelos.

### 1) Modelo de Random Forest.

In [ ]:
# Dividimos entre el texto y las etiquetas objetivo:
x_data = df['comment']
y_data = df['subreddit']

#### I) Vectorización de palabras:

In [ ]:
# Vectorizamos las palabras mediante el uso del método Bag of Words (BoW):
count_vectorizer = CountVectorizer()
x_counts= count_vectorizer.fit_transform(x_data)

#### II) Validación cruzada:

In [ ]:
# Dividimos el DataFrame en un 80% de entrenamiento y un 20% de testing:
x_train, x_test, y_train, y_test = train_test_split(x_counts, y_data, test_size=0.2, random_state=42)

#### III) Entrenamiento del modelo:

In [ ]:
# Usaremos el modelo clasificador de Random Forest, el cual entrenaremos a continuación:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(x_train, y_train)

RandomForestClassifier()

#### IV) Análisis de precisión:

In [ ]:
# Realizamos la predicción mediante los datos reservados para el testing y evaluamos
# su precisión en función de los valores reales (y_test):
y_pred = rf_classifier.predict(x_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy

0.2200557103064067

In [ ]:
# Parece ser que la precisión fue pésima. No obstante, no encontramos ningún error
# en la construcción de nuestro modelo.

### 2) Modelo alternativo.

In [ ]:
pass